# Creative Writing Test Conversation.ipynb

Experiment with a Creative Writing question and the self-refine method using the OpenAI API. 

In [1]:
import openai

# Get and set API key
with open('C:/Users/ijyli/Documents/OpenAI/anlp23-project.txt', 'r') as file:
    api_key = file.read()
openai.api_key = api_key


In [3]:
completion = openai.ChatCompletion.create(
  model="gpt-4-0613", 
  messages=[{"role": "user", "content": "What is the OpenAI mission?"}]
)

print(completion)


{
  "id": "chatcmpl-8IKvEhkvvaP5sgB27s7MCaffJ1mif",
  "object": "chat.completion",
  "created": 1699381020,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "OpenAI's mission is to ensure that artificial general intelligence (AGI) benefits all of humanity. They aim to directly build safe and beneficial AGI or to help others achieve these outcomes. OpenAI commits to utilize any influence over AGI's deployment to avoid harmful uses or undue concentration of power, and to prioritize long-term safety, technical leadership, and providing public goods to help society."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 14,
    "completion_tokens": 78,
    "total_tokens": 92
  }
}


AttributeError: text

In [13]:
print(completion['choices'][0]['message']['content'])


OpenAI's mission is to ensure that artificial general intelligence (AGI) benefits all of humanity. They aim to directly build safe and beneficial AGI or to help others achieve these outcomes. OpenAI commits to utilize any influence over AGI's deployment to avoid harmful uses or undue concentration of power, and to prioritize long-term safety, technical leadership, and providing public goods to help society.


In [25]:
# Send the current conversation as well as a new prompt and get a response
def prompt_gpt_4_and_get_convo(messages, prompt):
    messages_and_prompt = messages.copy()
    messages_and_prompt.append({"role": "user", "content": prompt})
    new_message = openai.ChatCompletion.create(
        model="gpt-4-0613", 
        messages=messages_and_prompt
    )
    with_new_message = messages_and_prompt.copy()
    with_new_message.append(dict(new_message['choices'][0]['message']))
    return with_new_message


In [31]:
conversation = []
with_response = prompt_gpt_4_and_get_convo(conversation, 'What is the longest river in the world?')
print(with_response)


[{'role': 'user', 'content': 'What is the longest river in the world?'}, {'role': 'assistant', 'content': 'The longest river in the world is the Nile River.'}]


In [32]:
with_another_response = prompt_gpt_4_and_get_convo(with_response, 'What continent is it on?')
print(with_another_response)


[{'role': 'user', 'content': 'What is the longest river in the world?'}, {'role': 'assistant', 'content': 'The longest river in the world is the Nile River.'}, {'role': 'user', 'content': 'What continent is it on?'}, {'role': 'assistant', 'content': 'The Nile River is in Africa.'}]


In [39]:
# Testing self-refine on a Creative Writing test question

# Load in text file data_100_random_text.txt
with open('data_100_random_text.txt', 'r') as file:
    data = file.read()

# Get first line
first_line = data.split('\n')[0]
print(first_line)

# Split on periods
sentences = first_line.split('.')
# print(sentences[0:2])

# Get back form with periods and also stripping leading and trailing blanks
sentences_with_periods = [sentence.strip() + '.' for sentence in sentences]
print(sentences_with_periods[0:2])


It isn't difficult to do a handstand if you just stand on your hands. It caught him off guard that space smelled of seared steak. When she didn’t like a guy who was trying to pick her up, she started using sign language. Each person who knows you has a different perception of who you are.
["It isn't difficult to do a handstand if you just stand on your hands.", 'It caught him off guard that space smelled of seared steak.']


In [42]:
# Initial task
test_task = "Task: Write a coherent passage of 2 short paragraphs. The end sentence of each paragraph must be: 1. " + sentences_with_periods[0] + " 2. " + sentences_with_periods[1]
test_task = test_task + "\nResponse: "

print(test_task)


Task: Write a coherent passage of 2 short paragraphs. The end sentence of each paragraph must be: 1. It isn't difficult to do a handstand if you just stand on your hands. 2. It caught him off guard that space smelled of seared steak.


In [43]:
# Initial ask
with_response = prompt_gpt_4_and_get_convo(conversation, test_task)

print(with_response)


[{'role': 'user', 'content': "Task: Write a coherent passage of 2 short paragraphs. The end sentence of each paragraph must be: 1. It isn't difficult to do a handstand if you just stand on your hands. 2. It caught him off guard that space smelled of seared steak."}, {'role': 'assistant', 'content': "Growing up, Timmy always considered himself quite the acrobat. His parents often would find him doing cartwheels in the garden, practicing somersaults on the living room rug, or trying to balance himself on the fence posts. Despite the occasional mishap, Timmy's quirky habits did much to entertain them. As Timmy became more confident in his abilities, he decided to challenge himself with a handstand. Eventually, he mastered it enough to demonstrate at a school talent show, causing a stir amongst his peers with his balancing prowess. It isn't difficult to do a handstand if you just stand on your hands.\n\nYears later, Timmy's adventurous nature sent him on to bigger and more ambitious endeav

In [45]:
print(with_response[-1]['content'])


Growing up, Timmy always considered himself quite the acrobat. His parents often would find him doing cartwheels in the garden, practicing somersaults on the living room rug, or trying to balance himself on the fence posts. Despite the occasional mishap, Timmy's quirky habits did much to entertain them. As Timmy became more confident in his abilities, he decided to challenge himself with a handstand. Eventually, he mastered it enough to demonstrate at a school talent show, causing a stir amongst his peers with his balancing prowess. It isn't difficult to do a handstand if you just stand on your hands.

Years later, Timmy's adventurous nature sent him on to bigger and more ambitious endeavors. His acrobat roots and thirst for a challenge led him to pursue a career as an astronaut. After years of rigorous training, he finally got the opportunity to go to space. He was prepared to face weightlessness and the mesmerizing view of Earth from above, but there was something nobody had warned h

In [47]:
# Self-refine prompts
self_refine_2 = "Your Task: Provide feedback on the correctness and coherence of the response and a rating on a scale of 1-10. If it is already coherent and correct to the extent you would award a 10, output 10 and the word STOP."
self_refine_2 = self_refine_2 + "\nFeedback: "
print(self_refine_2)

self_refine_3 = "Your Task: Rewrite the passage based on the most recent feedback."
self_refine_3 = self_refine_3 + "\nResponse: "
print(self_refine_3)


Your Task: Provide feedback on the correctness and coherence of the response and a rating on a scale of 1-10. If it is already coherent and correct to the extent you would award a 10, output 10 and the word STOP.
Feedback: 
Your Task: Rewrite the passage based on the most recent feedback.
Response: 


In [50]:
# Self-refinement loop
# Storing conversation elements
conversation = []
updated_convo = []
new_last_step = ""
# Maximum of 16 items in the conversation
for i in range(16):
    convo_to_feed = updated_convo.copy()
    last_step_this_loop = new_last_step
    # Get the response
    if i == 0:
        updated_convo = prompt_gpt_4_and_get_convo(conversation, test_task)
        new_last_step = "initial"
    #print(updated_convo)
    # If the response contains STOP, stop
    if "STOP" in updated_convo[-1]['content']:
        break
    # If last step was initial step or self_refine_3 need self_refine_2
    if last_step_this_loop == "initial" or last_step_this_loop == "self_refine_3":
        updated_convo = prompt_gpt_4_and_get_convo(convo_to_feed, self_refine_2)
        new_last_step = "self_refine_2"
    # If last step was tot_prompt_2 need tot_prompt_3
    if last_step_this_loop == "self_refine_2":
        updated_convo = prompt_gpt_4_and_get_convo(convo_to_feed, self_refine_3)
        new_last_step = "self_refine_3"


[{'role': 'user', 'content': "Task: Write a coherent passage of 2 short paragraphs. The end sentence of each paragraph must be: 1. It isn't difficult to do a handstand if you just stand on your hands. 2. It caught him off guard that space smelled of seared steak."}, {'role': 'assistant', 'content': 'For years, Bob had been practicing yoga, aiming to increase his balance and flexibility. He enjoyed the sense of peace it brought and aspired to conquer the most challenging poses. Up to this point, he had succeeded in perfecting various poses, but the handstand had always been a daunting challenge. He decided to simplify his approach. His yoga instructor once said, "In order to master a handstand, you must first learn to stand on your hands." Bob realized the truth in those words: it wasn\'t about having impressive strength or perfect balance. It wasn\'t difficult to do a handstand if you just stand on your hands.\n\nAfter his yoga session, Bob sat down for an afternoon of reading. Being a

In [51]:
print(updated_convo)


[{'role': 'user', 'content': "Task: Write a coherent passage of 2 short paragraphs. The end sentence of each paragraph must be: 1. It isn't difficult to do a handstand if you just stand on your hands. 2. It caught him off guard that space smelled of seared steak."}, {'role': 'assistant', 'content': 'For years, Bob had been practicing yoga, aiming to increase his balance and flexibility. He enjoyed the sense of peace it brought and aspired to conquer the most challenging poses. Up to this point, he had succeeded in perfecting various poses, but the handstand had always been a daunting challenge. He decided to simplify his approach. His yoga instructor once said, "In order to master a handstand, you must first learn to stand on your hands." Bob realized the truth in those words: it wasn\'t about having impressive strength or perfect balance. It wasn\'t difficult to do a handstand if you just stand on your hands.\n\nAfter his yoga session, Bob sat down for an afternoon of reading. Being a

In [15]:
# Output updated_convo to a text file
# Each element of the list should be on a new line
with open('updated_convo.txt', 'w') as file:
    for item in updated_convo:
        file.write("%s\n" % item)
